In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#### Check wording directory

In [10]:
# check working directory
import os
os.getcwd() # if directory is subfolder, change to home
os.chdir('/home/sukayna/Documents/github/newspaper')

In [2]:
import os
os.getcwd()
os.chdir('/Users/nhuquynh/Documents/Frontline/newspaper/')

#### Import packages

In [3]:
import json
import nltk
from nltk.collocations import *
from  nltk.text import ConcordanceIndex
import datetime as dt
import locale
import spacy
from tqdm import tqdm
import pprint
from nltk.text import Text
from nltk.tokenize import word_tokenize
nltk.download('punkt')

/Users/nhuquynh/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/nhuquynh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### Load data

In [4]:
# always use json to load the corpus
with open('data/factiva_data.json', 'r') as f:
    factiva_corpus = json.load(f)

#### Concordance analysis - keywords in context analysis 

In [5]:
# some quick tests
corpus = 'wiesbaden ist einen weiteren schritt gegangen um frauen und mädchen künftig besser vor gewalt zu schützen der ausschuss für frauen, gleichstellung und sicherheit hat einem antrag zugestimmt der vorsieht dass die stadt personal für die arbeit einer koordinierungsstelle finanzieren soll.'
tokens = word_tokenize(corpus)
mytext = Text(tokens)
mytext.concordance("frauen")

Displaying 2 of 2 matches:
 einen weiteren schritt gegangen um frauen und mädchen künftig besser vor gewa
ewalt zu schützen der ausschuss für frauen , gleichstellung und sicherheit hat


In [6]:
# turn documents into tokenized corpus
mycorpus = []
for doc in factiva_corpus:
    mycorpus.append(word_tokenize(doc['body']))

In [7]:
# concordance for first document
mytext = Text(mycorpus[0])
mytext.concordance("gewalt")

Displaying 3 of 3 matches:
[ 'es gibt physische gewalt , es gibt psychische gewalt - und e
ysische gewalt , es gibt psychische gewalt - und es gibt digitale gewalt . wie
ische gewalt - und es gibt digitale gewalt . wie diese wirkt , wenn sie vom st


In [8]:

def concordance(ci, word, width=75, lines=25):
    """
    Rewrite of nltk.text.ConcordanceIndex.print_concordance that returns results
    instead of printing them. 

    See:
    http://www.nltk.org/api/nltk.html#nltk.text.ConcordanceIndex.print_concordance
    """
    half_width = (width - len(word) - 2) // 2
    context = width // 4 # approx number of words of context

    results = []
    offsets = ci.offsets(word)
    if offsets:
        lines = min(lines, len(offsets))
        for i in offsets:
            if lines <= 0:
                break
            left = (' ' * half_width +
                    ' '.join(ci._tokens[i-context:i]))
            right = ' '.join(ci._tokens[i+1:i+context])
            left = left[-half_width:]
            right = right[:half_width]
            results.append('%s %s %s' % (left, ci._tokens[i], right))
            lines -= 1
    
    final_results = []
    for re in results:
        final_results.append(re.strip())

    return final_results

##### Concordance analysis for one term

In [9]:
# concordance for all documents for the term 'violence'
concordance_corpus = []
for article in mycorpus: 
    article_text = Text(article)
    ci = ConcordanceIndex(article_text)
    concordance_corpus.append(concordance(ci, 'gewalt'))

In [10]:
# look at first 5 results
for doc in concordance_corpus[:4]: 
    print(doc)

['gewalt , es gibt psychische gewalt - und', 'gewalt - und es gibt digitale gewalt . w', 'gewalt . wie diese wirkt , wenn sie vom']
['gewalt gegen frauen . sie zeigt : es dom', 's framing genutzt , sexualisierte gewalt sei ein problem , das von außen i', 'che einzelfälle ? wie medien über gewalt an frauen berichten “ ( klicken s', 'g an , schon einmal sexualisierte gewalt erfahren zu haben . natürlich spi', 'ewicht von realer und berichteter gewalt lässt sich trotz allem stark verm', 'ernüchterndes bild darüber , wie gewalt gegen frauen in den medien stattf', 'g verdient . zudem sind viele von gewalt betroffene aus verständlichen grü', 'est , dass die dokumentation über gewalt gegen frauen leicht gestiegen ist', 'icht nach misogyner struktureller gewalt . diese begriffe konnte meltzer n']
['anzeigenanstieg wegen häuslicher gewalt gegen frauen und kinder . das pro', "n ( grüne ) . ' , ' '' wir werden gewalt nur dann eindämmen können , wenn", "achbarn sind oft die ersten , die gewalt bem

Interesting unigrams:
- “Gewaltopfer” vs “Gewalttäter” => german words for victim and perpetrator of violence
- “Beziehungsgewalt” => not domestic but relationship violence
- “Partnerschaft” vs “Beziehung” => check for these synonyms
- “mutmaßlich[xx]” => police reporting

##### Concordance analysis for multiple terms

In [11]:
# concordance for all documents for the term 'domestic violence'

for article in mycorpus: 
    article_text = Text(article)
    article_text.concordance(['häusliche', 'gewalt'])

no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
esystem , vor allem im bereich häusliche gewalt . gerade hat saskia veit-prang
no matches
no matches
no matches
no matches
Displaying 7 of 7 matches:
estens 80 prozent an . oft ist häusliche gewalt nicht zu beweisen . die intimi
t darüber informiert sind , wo häusliche gewalt beginnt , weil sie vorurteilen
en partner selbst ausgesucht . häusliche gewalt ist somit oft auch ein struktu
dass sie als „ anwendung gegen häusliche gewalt “ erkennbar ist . die entwickl
altspirale brechen , die durch häusliche gewalt entsteht , und dass täter häuf
 hat das projekt initiiert . „ häusliche gewalt ist schon lange ein thema , we
on viel früher erfahren , dass häusliche gewalt nicht bei körperlicher gewalt 
no matches
no matches
no matches
no matches
Displaying 4 of 4 matches:
nd betrunken männer ertragen . häusliche gewalt und fußball sind leider brutal
ance service ( wmas ) einmal : häusliche gewalt nehme 

In [12]:
# don't run this, it takes too long and also unnecessary stuff such as no-matches

import sys
with open('file', 'w') as sys.stdout:
    for article in mycorpus: 
        article_text = Text(article)
        article_text.concordance(['häusliche', 'gewalt'])
    sys.close(1)